In [ ]:
import numpy as np
import numpy.linalg as la
from rum import matprint
PRECISION = 8
matformat = ".2f"

## Notation

### Voigt Notation (nicht-normiert)
![images/voigt1.png](images/voigt1.png)
![images/voigt1.png](images/voigt2.png)

2er kommen vom symmetrischen Teil

#### Switching between $\sigma$ to $\epsilon$ notation
$
A_\epsilon = D_2 A_\sigma
$

$
A_\sigma = D_{1/2} A_\epsilon^T 
$

#### Scalar product

$
\mathbf{A} \cdot \mathbf{B} = A_\epsilon^T B_\sigma = A_\sigma^T B_\epsilon =  A_\sigma^T D_2 B_\sigma = A_\epsilon^T D_{1/2} B_\epsilon
$

with 
$
D_\alpha = diag(1,1,1,\alpha,\alpha,\alpha)
$

#### Norm

$
\left| \left| \mathbf{A} \right| \right| = \sqrt{\mathbf{A} \cdot \mathbf{A}} = \sqrt{A_\sigma^T D_2 A_\sigma} = \sqrt{A_\sigma^T A_\epsilon} = \sqrt{A_\epsilon^T D_{1/2} A_\epsilon} 
$
### Mandel Nottation (normierte Voigt-Notation)

Idea: Vector notation for tensors, so that the tensor dot product can be calculated using the dot product of the vectors.

![images/mandel1.png](images/mandel1.png)

#### Dot (scalar) product
$
\mathbf{A} \cdot \mathbf{B} =  \left( {\hat A} \right) ^T \hat B
$

#### Norm

$
\left| \left| \mathbf{A} \right| \right| =  \sqrt{\left( {\hat A} \right) ^T \hat A} = \left| \left| \hat A \right| \right| _2
$

### Notation for 4th order Tensor C

#### Switching between Voigt and Mandel notation
$
\hat C = D_{\sqrt{2}} C D_{\sqrt{2}}
$

$
C = D_{\sqrt{2}/2} \hat C D_{\sqrt{2}/2}
$

#### Dot product
$
\mathbf{A} \cdot \mathbf{B} = tr {\hat A} \hat B = tr A D_4 B
$

#### Norm
$
\left| \left| \mathbf{A} \right| \right| =  \sqrt{A \cdot A } = \sqrt{tr \hat A \hat A } = \sqrt{trA D_4 A }
$

#### Eigenvalues
$
\hat C \hat \epsilon = D_{\sqrt{2}} C D_{\sqrt{2}} \hat \epsilon = \lambda \hat \epsilon
$

## Constitutive Laws (Voigt Notation)

In [ ]:
def CISO( K: float, G: float):
    """
    isotropic stiffness matrix in Voigt notation
    """
    lam=K-2/3*G;
    C = np.array([[lam+2*G, lam,     lam,     0,  0,  0],
                  [lam, lam+2*G,     lam,     0,  0,  0],
                  [lam,     lam, lam+2*G,     0,  0,  0],
                  [0,         0,       0,     G,  0,  0],
                  [0,         0,       0,     0,  G,  0],
                  [0,         0,       0,     0,  0,  G]])
    return C

In [ ]:
def CTISO(C1111, C3333, C2323, C1122, C1133):
    """
    transverse isotropic stiffness matrix in Voigt notation
    """
    C = np.array([[C1111, C1122,    C1133,      0,      0,              0],
                  [C1122, C1111,    C1133,      0,      0,              0],
                  [C1133, C1133,    C3333,      0,      0,              0],
                  [0,         0,        0,  C2323,      0,              0],
                  [0,         0,        0,      0,  C2323,              0],
                  [0,         0,        0,      0,      0, (C1111-C1122)/2]])
    return C

In [ ]:
def CCUBIC( C1111, C1122, C2323 ):
    """
    cubic symmetric stiffness matrix in Voigt notation
    """
    C = np.array([[C1111, C1122,  C1122,       0,      0,      0],
                  [C1122, C1111,  C1122,       0,      0,      0],
                  [C1122, C1122,  C1111,       0,      0,      0],
                  [    0,      0,      0,  C2323,      0,      0],
                  [    0,      0,      0,      0,  C2323,      0],
                  [    0,      0,      0,      0,      0,  C2323]])
    return C

## Functions

####  Functions to be applied to tensors in Mandel notation

In [ ]:
def scalar(A, B):
    return A.T.dot(B)

def deviator(A):
    trA = sum(A[0:3])
    return A - 1/3 * trA * np.array([1, 1, 1, 0, 0, 0])

def norm_fro(A):
    return np.sqrt(scalar(A,A))

####  Switching between Voigt and Mandel notation for 2nd order tensors in vector form

In [ ]:
def D(alpha):
    return np.array([1.,1.,1.,alpha,alpha,alpha])

In [ ]:
def sigma_to_eps(S):
    assert S.shape == (6,)
    alpha = 2.
    return D(alpha)*S
    
def eps_to_sigma(E):
    assert E.shape == (6,)
    alpha = 1./2.
    return D(alpha)*E

def sigma_to_mandel(S):
    assert S.shape == (6,), S.shape
    alpha = np.sqrt(2.0)
    return D(alpha)*S
    
def eps_to_mandel(E):
    assert E.shape == (6,)
    alpha = np.sqrt(2)/2.
    return D(alpha)*E

def mandel_to_eps(A_hat):
    assert A_hat.shape == (6,)
    alpha = 2./np.sqrt(2)
    return D(alpha)*A_hat

def mandel_to_sigma(A_hat):
    assert A_hat.shape == (6,)
    alpha = 1./np.sqrt(2)
    return D(alpha)*A_hat

####  Switching between Voigt and Mandel notation for 4th order tensors in matrix form
$
\hat C = D_{\sqrt{2}} C D_{\sqrt{2}}
$

$
C = D_{\sqrt{2}/2} \hat C D_{\sqrt{2}/2}
$

In [ ]:
def C_to_mandel(C):
    assert C.shape[0] == C.shape[1], "C is not square!"
    C_hat = C.copy()
    alpha = np.sqrt(2.0)
    for i in range(C.shape[0]):
            C_hat[i,:] = C_hat[i,:] * D(alpha)
            C_hat[:,i] = C_hat[:,i] * D(alpha)
    return C_hat


def Chat_to_voigt(C_hat):
    assert C_hat.shape[0] == C_hat.shape[1], "C is not square!"
    C = C_hat.copy()
    alpha = np.sqrt(2.0)/2.0
    for i in range(C.shape[0]):
            C[i,:] = C[i,:] * D(alpha)
            C[:,i] = C[:,i] * D(alpha)
    return C
    

In [ ]:
def StressStrain(C, E):
    """
    Linear stresses S as function of strains E, material C
    E and C must be given in non-normalized Voigt notation
    e.g. E = (E11,E22,E33,2E23,2E13,2E12)
    
    S is therefore:
    S = (S11,S22,S33,S23,S13,S12)
    """
    S = C.dot(E)
    return S

In [ ]:
def SigmaMises(S):
    """
    von Mises stress for a given stress vector S
    S must be given in non-normalized Voigt notation
    i.e. S = (S11,S22,S33,S23,S13,S12)
    """
    # convert to Mandel notation
    S = sigma_to_mandel(S)
    rt32 = np.sqrt(3/2)
    devS = deviator(S)
    return rt32*norm_fro(devS)

### Some Tests

In [ ]:
C = CISO(210e3, 0.3)
# E in Voigt Notation
E = np.array([1, 0, 0., 0., 0., 0.])
S=StressStrain(C, E)
print(S)

In [ ]:
S = np.array([-1260,800,0,0,0,500])
assert round(SigmaMises(S),6) == 1996.396754

In [ ]:
C = np.ones([6,6])
assert Chat_to_voigt(C_to_mandel(C)).all() == C.all()

## Spectral decomposition
### Function

In [ ]:
def SpectralDecomposition(C):
    assert np.allclose(C, C.T, atol=1**(-PRECISION)), "C is not symmetric"
    
    # Transformation Voigt -> Mandel
    C_hat = C_to_mandel(C)
    
    # EW,EV
    lambdas, vecs = la.eig(C_hat)
    lambdas = lambdas.round(PRECISION)
    assert lambdas.all() > 0, "C is not positive definite"
    
    P = []
    s_lambdas =  []

    for i in range(len(lambdas)):
        y = lambdas[i]
        v = vecs[i]
        if y in s_lambdas:
            j = s_lambdas.index(y)
            w = v - P[j].dot(v)
            #assert np.inner(w,v) == 0, "v and w should be orthogonal"
            w = w / la.norm(w, 2)
            P[j] = P[j] + np.outer(w,w)
        else:
            s_lambdas.append(y)
            w = np.outer(v,v)
            w = w / la.norm(w, 2)
            P.append(w)
            
    #Transformation Mandel -> Voigt
    P = [Chat_to_voigt(p) for p in P]
        
    return s_lambdas, P

### Verification
Reconstruction of stiffness matrix:
$
C = \sum_{i} \lambda_i P_i
$

In [ ]:
C = CISO(210e3, 0.3)
lambdas, P = SpectralDecomposition(C)
C_new = sum([lambdas[i]*P[i] for i in range(len(lambdas))])
print("C original:")
matprint(C, fmt=matformat)
print("C from projectors:")
matprint(C_new, fmt=matformat)

In other words, something's not right. But I can't figure out what it is... going to ask in class.
